In [4]:
from dotenv import load_dotenv

load_dotenv('../env')

True

In [7]:
from langchain_ollama import ChatOllama

base_url = "http://localhost:11434"
model = 'llama3.2:1b'
llm = ChatOllama(
    base_url=base_url,
    model = model,
    temperature = 0.8,
    num_predict = 256
)

In [14]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda, RunnableParallel
from langchain.output_parsers import (
    CommaSeparatedListOutputParser, 
    StructuredOutputParser, 
    ResponseSchema
)

# Create output parsers
fraud_detection_parser = StructuredOutputParser.from_response_schemas([
    ResponseSchema(
        name="risk_level", 
        description="Fraud risk level (low/medium/high)"
    ),
    ResponseSchema(
        name="reasoning", 
        description="Brief explanation of fraud risk assessment"
    )
])

compliance_check_parser = CommaSeparatedListOutputParser()

# Modify prompts to include format instructions
fraud_detection_prompt = ChatPromptTemplate.from_template(
    "Analyze the payment transaction for fraud risk:\n"
    "Amount: ${amount}\n"
    "Merchant: {merchant}\n"
    "Card Type: {card_type}\n\n"
    "Provide a detailed fraud assessment in STRICT JSON format. IMPORTANT: Ensure the output is a valid JSON object with EXACTLY these keys:\n"
    "{format_instructions}\n"
    "EXAMPLE FORMAT: {{\n"
    "  \"risk_level\": \"low\",\n"
    "  \"reasoning\": \"Detailed explanation here\"\n"
    "}}"
).partial(format_instructions=fraud_detection_parser.get_format_instructions())

compliance_check_prompt = ChatPromptTemplate.from_template(
    "Perform a compliance check on this payment transaction. "
    "Payment amount: ${amount}, Merchant: {merchant}, Country: {country}\n"
    "List any compliance concerns. Respond in a comma-separated list:\n"
    "{format_instructions}"
).partial(format_instructions=compliance_check_parser.get_format_instructions())

# Validation function using a simple string output parser
def validate_payment(payment_info):
    if payment_info['amount'] <= 0:
        return "Validation Failed: Invalid payment amount"
    return "Validation Passed"

# Payment processing chain using RunnableParallel
payment_processing_chain = RunnableParallel({
    # Validation step
    "validation": RunnableLambda(validate_payment),
    
    # Fraud detection using LLM with output parser
     "fraud_detection": fraud_detection_prompt | llm | fraud_detection_parser,
    
    # Compliance check using LLM with output parser
     "compliance_check": compliance_check_prompt | llm | compliance_check_parser
})

# Example usage
payment_info = {
    "amount": 5000,
    "merchant": "Global Tech Solutions",
    "card_type": "Corporate Credit Card",
    "country": "United States"
}

# Invoke the chain
result = payment_processing_chain.invoke(payment_info)
print("Payment Processing Results:")
for key, value in result.items():
    print(f"{key.replace('_', ' ').title()}: {value}")


OutputParserException: Got invalid JSON object. Error: Expecting value: line 1 column 1 (char 0)
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE 